In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.contour as cnt
import matplotlib.artist.Artist.get_visible

import scipy.stats
import scipy.optimize as optimize
from random import uniform
from matplotlib.widgets import Slider

In [2]:
sigma_xxyy = pd.read_csv('dataset/sigma_x-sigma_y.csv', sep=',', names=['sxx', 'syy'])
sxx = sigma_xxyy['sxx']
syy = sigma_xxyy['syy']

In [3]:
def yld2000(sigma_x, sigma_y, alpha, a):
    (alpha1, alpha2, alpha3, alpha4, alpha5, alpha6, alpha7, alpha8) = alpha
    phi1 = abs(((2*alpha1+alpha2)/3)*sigma_x - ((2*alpha2+alpha1)/3)*sigma_y)**a
    phi2 = abs(((2*alpha3-2*alpha4)/3)*sigma_x + ((4*alpha4-alpha3)/3)*sigma_y)**a + abs(((4*alpha5-alpha6)/3)*sigma_x + ((2*alpha6-2*alpha5)/3)*sigma_y)**a
    
    return ((phi1+phi2)/2)**(1/a)

In [33]:
def z_of_contour(ax, xmesh, ymesh, alpha, a, points, colors, ys):
    z = []
    for i in range(points):
        temp = []
        for j in range(points):
            temp.append(yld2000(xmesh[i,j], ymesh[i,j], alpha, a))
        z.append(temp)
    z = np.array(z)

    ax.contour(xmesh, ymesh, z-ys, levels=[1], colors=colors)

alpha = np.array([0.9349, 1.0671, 0.9690, 0.9861, 1.0452, 0.9999, 0.9097, 1.0805])
# alpha = np.array([1, 1, 1, 1, 1, 1, 1, 1])
phi = np.linspace(0, 2 * np.pi, 100)
sxx_sample = np.linspace(-200,200,500)
syy_sample = np.linspace(-200,200,500)
sx, sy = np.meshgrid(sxx, syy)
sx_sample, sy_sample = np.meshgrid(sxx_sample, syy_sample)
ys = 100
a = 6

%matplotlib widget

fig , ax = plt.subplots(1,1)

random_x = np.linspace(-200,200,len(sxx_sample))
random_z = np.linspace(-200,200,len(sxx_sample))
xx, zz = np.meshgrid(random_x, random_z)
ax.contour(xx, np.zeros(len(sxx_sample)**2).reshape(xx.shape), zz, levels=[1], colors='yellow')

random_y = np.linspace(-200,200,len(sxx_sample))
yy, zz = np.meshgrid(random_y, random_z)
ax.contour(np.zeros(len(sxx_sample)**2).reshape(xx.shape), yy, zz, levels=[1], colors='orange')

z_of_contour(ax, sx, sy, alpha, a, len(sxx), 'green', ys)
z_of_contour(ax, sx_sample, sy_sample, alpha, a, len(sxx_sample), 'red', ys)


labels = ['|| to sx', '|| to sy', 'Experimental Data', 'Random Data']
for i in range(len(labels)):
    ax.collections[i].set_label(labels[i])

ax.legend(loc='upper left')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Calculation of anisotropic parameters

Reference: Hamzah Abdulrahman Alharthi, *Accurate descriptions of the anisotropic plastic yielding behaviour of various metallic sheets*, Chapter 3 Page 31, Equation 3.16 

$$ F_{1} = \left\lvert{\alpha_{1} \frac{2}{3} - \alpha_{2} \left(\frac{-1}{3}\right)}\right \rvert^a  + \left\lvert{\alpha_{3} \frac{2}{3} + 2\alpha_{4} \left(\frac{-1}{3}\right)}\right \rvert^a + \left\lvert{2\alpha_{5} \frac{2}{3} + \alpha_{6} \left(\frac{-1}{3}\right)}\right \rvert^a - 2\left(\frac{\overline{\sigma}}{\sigma_{0}}\right)^a = 0 \tag{1} $$

$$ F_{2} = \left\lvert{\alpha_{1} \frac{-1}{3} - \alpha_{2} \left(\frac{2}{3}\right)}\right \rvert^a  + \left\lvert{\alpha_{3} \frac{-1}{3} + 2\alpha_{4} \left(\frac{2}{3}\right)}\right \rvert^a + \left\lvert{2\alpha_{5} \frac{-1}{3} + \alpha_{6} \left(\frac{2}{3}\right)}\right \rvert^a - 2\left(\frac{\overline{\sigma}}{\sigma_{90}}\right)^a = 0 \tag{2} $$

$$ F_{3} = \left\lvert{\alpha_{1} \frac{-1}{3} - \alpha_{2} \left(\frac{-1}{3}\right)}\right \rvert^a  + \left\lvert{\alpha_{3} \frac{-1}{3} + 2\alpha_{4} \left(\frac{-1}{3}\right)}\right \rvert^a + \left\lvert{2\alpha_{5} \frac{-1}{3} + \alpha_{6} \left(\frac{-1}{3}\right)}\right \rvert^a - 2\left(\frac{\overline{\sigma}}{\sigma_{b}}\right)^a = 0 \tag{3} $$

$$ G_{1} = \left(\alpha_{1}(1-R_{0}) + \alpha_{2}(2+R_{0})\right)\left(\alpha_{1} \frac{2}{3} - \alpha_{2} \left(\frac{-1}{3}\right)\right)\left\lvert{\alpha_{1} \frac{2}{3} - \alpha_{2} \left(\frac{-1}{3}\right)}\right \rvert^{a-2} +
\left(\alpha_{3}(1-R_{0}) - 2\alpha_{4}(2+R_{0})\right)\left(\alpha_{3} \frac{2}{3} + 2\alpha_{4} \left(\frac{-1}{3}\right)\right)\left\lvert{\alpha_{3} \frac{2}{3} + 2\alpha_{4} \left(\frac{-1}{3}\right)}\right \rvert^{a-2} +
\left(2\alpha_{5}(1-R_{0}) - \alpha_{6}(2+R_{0})\right)\left(2\alpha_{5} \frac{2}{3} + \alpha_{6} \left(\frac{-1}{3}\right)\right)\left\lvert{2\alpha_{5} \frac{2}{3} + \alpha_{6} \left(\frac{-1}{3}\right)}\right \rvert^{a-2} = 0 \tag{4} $$

$$ G_{2} = \left(\alpha_{1}(2+R_{90}) + \alpha_{2}(1-R_{90})\right)\left(\alpha_{1} \frac{-1}{3} - \alpha_{2} \left(\frac{2}{3}\right)\right)\left\lvert{\alpha_{1} \frac{-1}{3} - \alpha_{2} \left(\frac{2}{3}\right)}\right \rvert^{a-2} +
\left(\alpha_{3}(2+R_{90}) - 2\alpha_{4}(1-R_{90})\right)\left(\alpha_{3} \frac{-1}{3} + 2\alpha_{4} \left(\frac{2}{3}\right)\right)\left\lvert{\alpha_{3} \frac{-1}{3} + 2\alpha_{4} \left(\frac{2}{3}\right)}\right \rvert^{a-2} +
\left(2\alpha_{5}(2+R_{90}) - \alpha_{6}(1-R_{90})\right)\left(2\alpha_{5} \frac{-1}{3} + \alpha_{6} \left(\frac{2}{3}\right)\right)\left\lvert{2\alpha_{5} \frac{-1}{3} + \alpha_{6} \left(\frac{2}{3}\right)}\right \rvert^{a-2} = 0 \tag{5} $$

$$ G_{3} = \left(\alpha_{1}(1+2R_{b}) + \alpha_{2}(2+R_{b})\right)\left(\alpha_{1} \frac{-1}{3} - \alpha_{2} \left(\frac{-1}{3}\right)\right)\left\lvert{\alpha_{1} \frac{-1}{3} - \alpha_{2} \left(\frac{-1}{3}\right)}\right \rvert^{a-2} +
\left(\alpha_{3}(1+2R_{b}) - 2\alpha_{4}(2+R_{b})\right)\left(\alpha_{3} \frac{-1}{3} + 2\alpha_{4} \left(\frac{-1}{3}\right)\right)\left\lvert{\alpha_{3} \frac{-1}{3} + 2\alpha_{4} \left(\frac{-1}{3}\right)}\right \rvert^{a-2} +
\left(2\alpha_{5}(1+2R_{b}) - \alpha_{6}(2+R_{b})\right)\left(2\alpha_{5} \frac{-1}{3} + \alpha_{6} \left(\frac{-1}{3}\right)\right)\left\lvert{2\alpha_{5} \frac{-1}{3} + \alpha_{6} \left(\frac{-1}{3}\right)}\right \rvert^{a-2} = 0 \tag{6} $$

In [34]:
r0 = 1.56
r90 = 1.71
rb = 1.11

sigma0 = 1
sigma90 = 0.995
sigmab = 1.004

parameters = {
    1: {
        'strength_ratio': sigma0,
        'gamma': 2/3,
        'delta': -1/3,
        'p': 1-r0,
        'q': 2+r0,
    },
    2: {
        'strength_ratio': sigma90,
        'gamma': -1/3,
        'delta': 2/3,
        'p': 2+r90,
        'q': 1-r90,
    },
    3: {
        'strength_ratio': sigmab,
        'gamma': -1/3,
        'delta': -1/3,
        'p': 1+2*rb,
        'q': 2+rb,
    },
}

def F(gamma_j, delta_j, sigma_bar_sigma_j, alpha, a):
    alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = alpha
    x1 = abs(alpha1 * gamma_j - alpha2 * delta_j)**a
    x2 = abs(alpha3 * gamma_j + 2*alpha4 * delta_j)**a
    x3 = abs(2*alpha5 * gamma_j + alpha6 * delta_j)**a
    return x1 + x2 + x3 - 2*(sigma_bar_sigma_j)**a

def G(pj, qj, gamma_j, delta_j, alpha, a):
    alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = alpha
    x1 = (pj*alpha1 + qj*alpha2)*abs(alpha1 * gamma_j - alpha2 * delta_j)**(a-1)*np.sign(alpha1 * gamma_j - alpha2 * delta_j)
    x2 = (pj*alpha3 - 2*qj*alpha4)*abs(alpha3 * gamma_j + 2*alpha4 * delta_j)**(a-1)*np.sign(alpha3 * gamma_j + 2*alpha4 * delta_j)
    x3 = (2*pj*alpha5 - qj*alpha6)*abs(2*alpha5 * gamma_j + alpha6 * delta_j)**(a-1)*np.sign(2*alpha5 * gamma_j + alpha6 * delta_j)
    return x1 + x2 + x3

a = 6

def func(alpha):
    print(alpha)
    yield_strength_eqs = [F(parameters[i]['gamma'], parameters[i]['delta'], parameters[i]['strength_ratio'], alpha, a) for i in parameters]
    r_eqs = [G(parameters[i]['p'], parameters[i]['q'], parameters[i]['gamma'], parameters[i]['delta'], alpha, a) for i in parameters]
    return yield_strength_eqs + r_eqs
    
result = optimize.root(func, [1,1,1,1,1,1], method='hybr')
alpha = result.x
print(f'alpha: {alpha}')

[1 1 1 1 1 1]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
[1.00000001 1.         1.         1.         1.         1.        ]
[1.         1.00000001 1.         1.         1.         1.        ]
[1.         1.         1.00000001 1.         1.         1.        ]
[1.         1.         1.         1.00000001 1.         1.        ]
[1.         1.         1.         1.         1.00000001 1.        ]
[1.         1.         1.         1.         1.         1.00000001]
[1.01771197 1.07454987 1.1146693  0.97955752 0.97341386 1.00362923]
[1.021693   1.05534674 1.12641936 0.97739812 0.96954124 0.99721116]
[1.02050134 1.05891528 1.12910924 0.97703538 0.96861535 0.99474354]
[1.02079894 1.05862928 1.12956513 0.9769867  0.96841597 0.99404784]
[1.02077749 1.05859076 1.12954943 0.97699314 0.96840995 0.99398668]
[1.020783   1.05859388 1.12954476 0.97699419 0.96841007 0.9939837 ]
[1.02078229 1.05859398 1.1295435  0.97699444 0.96841026 0.99398387]
[1.0207823  1.05859403 1.12954335 0.97699447 0.96841027 0.9939